In [1]:
import inro.modeller as m
import os
import glob

mm = m.Modeller()
eb = m.Modeller().emmebank

#tools
NAMESPACE = "tmg.input_output.import_binary_matrix"
import_matrices = mm.tool(NAMESPACE)           
NAMESPACE = "inro.emme.data.matrix.change_matrix_properties"
change_matrix = m.Modeller().tool(NAMESPACE)
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
NAMESPACE = "inro.emme.data.matrix.export_matrix_to_csv"
export_matrices = m.Modeller().tool(NAMESPACE)

#HDR Scripts
common = mm.module('org.emme.common')
moe = mm.module('org.emme.moe')
sc_num = 114

#directory = "P:\\451010\\281829_GGH\\GGHM Runs\\2041\\GGHM Base Case\\"
#directory = "P:\\451010\\281829_GGH\\GGHM Runs\\2016\\"
#directory = "P:\\451010\\281829_GGH\\GGHM Runs\\2011 GGHM validation run\\"
#directory = "P:\\451010\\281829_GGH\\GGHM Runs\\2041\\MTO_2041_GGH_MTP_302_V103_CS221_20180601\\"
directory = "P:\\451010\\281829_GGH\\GGHM Runs\\2041\\MTP_304\\"

pop_o_matrix = "mo1"
emp_o_matrix = "mo2"
#senior_o_matrix = "mo3"
#child_o_matrix = "mo4"
#low_inc_o_matrix = "mo5"
#youth_o_matrix = "mo6"
pop_d_matrix = "md1"
emp_d_matrix = "md2"


In [2]:
#function definitons

def non_zero_only (string):
    return "((" + string + ")==0)+(" + string + ")"

def new_matrix_name(m_id, name):
    if not(eb.matrix(m_id)):
        create_matrix(matrix_id = m_id,
            matrix_name = name,
            matrix_description = name)
    else:
        change_matrix(m_id, name, name)
        
def calc_avg_od (expression_string, denom_nums, result_num, result_name):

    #final origin matrix
    denom_string = ""
    for matrix in denom_nums:
        denom_string += "mo" + matrix + "+"
    denom_string = denom_string[:-1]
    o_matrix = "mo" + result_num
    new_matrix_name(o_matrix, result_name)
    
    #temp origin matrix
    temp_o_matrix = eb.available_matrix_identifier('ORIGIN')
    new_matrix_name(temp_o_matrix, "")
    spec = common.get_mc_spec(temp_o_matrix, expression_string, None, "+",None, None, None)
    matrix_calculator(spec)
    spec = common.get_mc_spec(o_matrix, temp_o_matrix + "/(" + denom_string + ")", None, None,None, None, None)
    matrix_calculator(spec)

    #final destination matrix
    denom_string = ""
    for matrix in denom_nums:
        denom_string += "md" + matrix + "+"
    denom_string = denom_string[:-1]
    d_matrix = "md" + result_num
    new_matrix_name(d_matrix, result_name)
    
    #temp destination matrix
    temp_d_matrix = eb.available_matrix_identifier('DESTINATION')
    new_matrix_name(temp_d_matrix, "")
    spec = common.get_mc_spec(temp_d_matrix, expression_string, "+", None,None, None, None)
    matrix_calculator(spec)
    
    spec = common.get_mc_spec(d_matrix, temp_d_matrix + "/(" + denom_string + ")", None, None,None, None, None)
    matrix_calculator(spec)

    eb.delete_matrix(temp_o_matrix)
    eb.delete_matrix(temp_d_matrix)

In [3]:
#delete all FULL matrices
common.clear_matrices(eb)

In [4]:
#matrix definitions
sov_num = "mf13"
hov2_num = "mf14"
hov3_num = "mf15"
tran_num = "mf16"
bike_num = "mf17"
walk_num = "mf18"
truck_num = "mf19"
sov_dist = "mf1"
sov_time = "mf2"
hov2_dist = "mf3"
hov2_time = "mf4"
hov3_dist = "mf5"
hov3_time = "mf6"
min_transit_time = "mf20"

#averaged matrices 
avg_auto_time = "mf7"
avg_auto_dist = "mf8"
avg_sov_time = "mf9"
avg_hov_time = "mf10"
avg_tran_time = "mf11"
avg_active_time = "mf12"
avg_truck_time = "mf21"
avg_trip_time = "mf22"
avg_trip_dist= "mf23"
avg_hov_dist = "mf24"
avg_tran_dist = "mf25"
vkt_mtx = "mf26"
pkt_mtx = "mf27"

pop_tran_mtx = "mf28"
emp_tran_mtx = "mf29"
pop_emp_tran_mtx = "mf30"

pop_auto_mtx = "mf31"
emp_auto_mtx = "mf32"
pop_emp_auto_mtx = "mf33"

pop_active_mtx = "mf34"
emp_active_mtx = "mf35"
pop_emp_active_mtx = "mf36"

In [5]:
# calculate minimum transit time
common.minimum_transit_time("peak", min_transit_time, directory ,sc_num)
common.clear_matrices(eb,[min_transit_time])

Imported matrix mf3
Imported matrix mf19
Imported matrix mf6
Imported matrix mf15
Imported matrix mf4
Imported matrix mf24
Imported matrix mf17
Imported matrix mf11
Imported matrix mf10
Imported matrix mf12
Imported matrix mf8
Imported matrix mf5
Imported matrix mf14
Imported matrix mf16
Imported matrix mf13
Imported matrix mf22
Imported matrix mf23
Imported matrix mf21
Imported matrix mf1
Imported matrix mf2
Imported matrix mf9
Imported matrix mf7
Imported matrix mf18
Finished Importing Matrices
Finished Calculating Transit Travel Times


In [7]:
#calculate total truck demand
truck_list = {'lt_truck_ampkhr':"",
               'mt_truck_ampkhr':"",
               'ht_truck_ampkhr':"",
            }

for matrix_name in truck_list.keys():
    truck_list[matrix_name] = eb.available_matrix_identifier('FULL')
    create_matrix(matrix_id = truck_list[matrix_name],
                      matrix_name= matrix_name,
                      matrix_description = matrix_name)
common.import_matrices_from_directory(directory, truck_list, ".mdf", sc_num)
new_matrix_name(truck_num, "truck.trips")
expression = "(" + truck_list['lt_truck_ampkhr'] + "+" + truck_list['mt_truck_ampkhr'] + "+" +truck_list['ht_truck_ampkhr']+ ")*5" 
spec = common.get_mc_spec(truck_num, expression, None, None,None, None, None)
matrix_calculator(spec)
common.clear_matrices(eb,[min_transit_time,truck_num])

Imported matrix mf1
Imported matrix mf3
Imported matrix mf2
Matrix ht_truck_ampkhr was not imported, or contains no values!
Finished Importing Matrices


In [8]:
#peak demand matrices

matrix_list={'auto_sov_PkPd':sov_num,
             'auto_hov2_PkPd':hov2_num,
             'auto_hov3_PkPd':hov3_num,
             'PnA_Transit_PkPd':tran_num,
             'Bike_PkPd':bike_num,
             'Walk_PkPd':walk_num,
             'truck.trips': truck_num
            }

In [ ]:
#off-peak demand matrices

matrix_list={'auto_sov_offPkPd':sov_num,
             'auto_hov2_offPkPd':hov2_num,
             'auto_hov3_offPkPd':hov3_num,
             'PnA_Transit_offPkPd':tran_num,
             'Bike_offPkPd':bike_num,
             'Walk_offPkPd':walk_num
            }

In [12]:
#peak skims
matrix_list={'skim.am_peak.auto.sov.dist' : sov_dist,
             'skim.am_peak.auto.sov.time' : sov_time,
             'skim.am_peak.auto.hov2.dist' : hov2_dist,
             'skim.am_peak.auto.hov2.time' : hov2_time,
             'skim.am_peak.auto.hov3.dist' : hov3_dist,
             'skim.am_peak.auto.hov3.time' : hov3_time,
            }

In [ ]:
#off-peak skims
matrix_list={'skim.midday.auto.sov.dist' : sov_dist,
             'skim.midday.auto.sov.time' : sov_time,
             'skim.midday.auto.hov2.dist' : hov2_dist,
             'skim.midday.auto.hov2.time' : hov2_time,
             'skim.midday.auto.hov3.dist' : hov3_dist,
             'skim.midday.auto.hov3.time' : hov3_time
            }

In [25]:
#peak demand matrices

matrix_list={'trips_peak_sov':sov_num,
             'trips_peak_hov2':hov2_num,
             'trips_peak_hov3pl':hov3_num,
             'trips_transit_peak':tran_num, 
             'trips_peak_walk':walk_num,
             'trips_peak_bike': bike_num,
             'truck.trips': truck_num
            }

In [12]:
#peak skims
matrix_list={'skim.am_peak.auto.sov.dist' : sov_dist,
             'skim.am_peak.auto.sov.time' : sov_time,
             'skim.am_peak.auto.hov2.dist' : hov2_dist,
             'skim.am_peak.auto.hov2.time' : hov2_time,
             'skim.am_peak.auto.hov3.dist' : hov3_dist,
             'skim.am_peak.auto.hov3.time' : hov3_time, 
            }

In [26]:
### IMPORT ALL MATRICES IN SUBFOLDERS ###
common.import_matrices_from_directory(directory, matrix_list, ".mdf", sc_num)
for matrix in matrix_list:
    num = matrix_list[matrix]
    change_matrix(num ,matrix,matrix)

Imported matrix mf17
Imported matrix mf14
Imported matrix mf15
Imported matrix mf13
Imported matrix mf18
Imported matrix mf16
Finished Importing Matrices


In [27]:
# convert full matrices to O and D matrices - demand only

for demand_matrix in matrix_list:
    full_id = matrix_list[demand_matrix]
    origin_id = "mo" + full_id[2:]
    destin_id = "md" + full_id[2:]
    new_matrix_name(origin_id, "total." + demand_matrix)
    spec = common.get_mc_spec(origin_id, full_id,None, "+" , None, None, None)
    matrix_calculator(spec)
    new_matrix_name(destin_id, "total." + demand_matrix)
    spec = common.get_mc_spec(destin_id, full_id,"+",None , None, None, None)
    matrix_calculator(spec)

In [28]:
#Travel times
#1.	Average transit trip travel times by origin and destination zone

#get total travel time
expression_string = tran_num + "*" + min_transit_time
denom_nums = [tran_num[2:]]

calc_avg_od (expression_string, [tran_num[2:]], avg_tran_time[2:], "avg.transit.time")


#2.	Average HOV travel time by origin and destination zone

#get total travel time
expression_string = hov2_num + "*" + hov2_time+ "+" + hov3_num + "*" + hov3_time

calc_avg_od (expression_string, [hov2_num[2:],hov3_num[2:]], avg_hov_time[2:], "avg.hov.time")

#3.	Average truck travel times by origin and destination zone

#get total travel time
expression_string =  truck_num + "*" + sov_time

calc_avg_od (expression_string, [truck_num[2:]], avg_truck_time[2:], "avg.truck.time")

#4.	Average travel time by origin and destination for all person modes (no trucks)

expression_string =  sov_num + "*" + sov_time + "+" +\
    hov2_num + "*" + hov2_time + "+" +\
    hov3_num + "*" + hov3_time + "+" +\
    tran_num + "*" + min_transit_time + "+" +\
    bike_num + "*" + sov_dist + "*4+" +\
    walk_num + "*" + sov_dist + "*15"

calc_avg_od (expression_string, [sov_num[2:],hov2_num[2:],hov3_num[2:],tran_num[2:],bike_num[2:],walk_num[2:]], \
             avg_trip_time[2:], "avg.trip.time")

#Trip distance
#5.	Average trip distance by origin and destination for all person trips (no trucks)

expression_string =  "(" + sov_num + "+" + hov2_num + "+" + hov3_num + "+" +\
    tran_num + "+" + bike_num + "+" + walk_num + ")*" + sov_dist

calc_avg_od (expression_string, [sov_num[2:],hov2_num[2:],hov3_num[2:],tran_num[2:],bike_num[2:],walk_num[2:]], \
             avg_trip_dist[2:], "avg.trip.dist")

#6.	Average trip distance by origin and destination for HOV trips

expression_string =  "(" + hov2_num + "+" + hov3_num + ")*" + sov_dist

calc_avg_od (expression_string, [hov2_num[2:],hov3_num[2:]], \
             avg_hov_dist[2:], "avg.hov.dist")

#7.	Average trip distance by origin and destination for transit trips
expression_string =  "(" + tran_num + ")*" + sov_dist

calc_avg_od (expression_string, [tran_num[2:]], \
             avg_tran_dist[2:], "avg.transit.dist")

#8.	PKT by origin and destination

new_matrix_name("mo" + pkt_mtx[2:] , "total.pkt")
expression = sov_dist + "*(" +tran_num + "+" + bike_num + "+" + walk_num+ "+" + sov_num \
    + "+" + hov2_num + "*2.2+" + hov3_num + "*3.3)"
spec = common.get_mc_spec("mo" + pkt_mtx[2:], expression , None,"+",None, None, None)
matrix_calculator(spec)

new_matrix_name("md" + pkt_mtx[2:] , "total.pkt")
expression = sov_dist + "*(" +tran_num + "+" + bike_num + "+" + walk_num+ "+" + sov_num \
    + "+" + hov2_num + "*2.2+" + hov3_num + "*3.3)"
spec = common.get_mc_spec("md" + pkt_mtx[2:], expression , "+",None,None, None, None)
matrix_calculator(spec)

#9.	VKT by origin and destination

new_matrix_name("mo" + vkt_mtx[2:] , "total.vkt")
expression = sov_dist + "*(" + sov_num + "+" + hov2_num + "+" + hov3_num + ")"
spec = common.get_mc_spec("mo" + vkt_mtx[2:], expression , None,"+",None, None, None)
matrix_calculator(spec)

new_matrix_name("md" + vkt_mtx[2:] , "total.vkt")
expression = sov_dist + "*(" + sov_num + "+" + hov2_num + "+" + hov3_num + ")"
spec = common.get_mc_spec("md" + vkt_mtx[2:], expression , "+",None,None, None, None)
matrix_calculator(spec)


{'average': 14.772054,
 'expression_type': 'FULL',
 'maximum': 41423.164062,
 'maximum_at': {'destination': 9010, 'origin': 9013},
 'minimum': 0.0,
 'minimum_at': {'destination': 1001, 'origin': 1001},
 'num_evaluations': 12645136,
 'result_type': 'DESTINATION',
 'sum': 186794624.0}

In [29]:
spec = common.get_mc_spec(None,pop_o_matrix,"+",None,None,None,None)
total_pop = matrix_calculator(spec)['result']

spec = common.get_mc_spec(None,emp_o_matrix,"+",None,None,None,None)
total_emp = matrix_calculator(spec)['result']

#Accessibility 
#For these accessibility measures by zone, just want to confirm that we are including population and employment within the zone as well.
#10.	% of population accessible by transit in under 45 minutes by zone

expression_string =  "("+ pop_d_matrix + "*(" + min_transit_time + "<45))/" + str(total_pop)
new_matrix_name("mo" + pop_tran_mtx[2:], "access.pop.transit")
spec = common.get_mc_spec("mo" + pop_tran_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)

#11.	% of jobs accessible by transit in under 45 minutes by zone

expression_string =  "("+ emp_d_matrix + "*(" + min_transit_time + "<45))/" + str(total_emp)
new_matrix_name("mo" + emp_tran_mtx[2:], "access.emp.transit")
spec = common.get_mc_spec("mo" + emp_tran_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)

#12.	% of people+jobs accessible by transit in under 45 minutes by zone

expression_string =  "(("+ emp_d_matrix + " +" + pop_d_matrix + ")*(" + min_transit_time + "<45))/" + str(total_emp+total_pop)
new_matrix_name("mo" + pop_emp_tran_mtx[2:], "access.pop.emp.transit")
spec = common.get_mc_spec("mo" + pop_emp_tran_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)
#(For 8,9,10 if possible we need for all, and then broken down for seniors, youth and low income)

#13.	% of people accessible by road (SOV+HOV) in under 45 min by zone

expression_string =  "("+ pop_d_matrix + "*(" + hov3_time + "<45))/" + str(total_pop)
new_matrix_name("mo" + pop_auto_mtx[2:], "access.pop.auto")
spec = common.get_mc_spec("mo" + pop_auto_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)

#14.	 % of jobs accessible by road (SOV+HOV) in under 45 min by zone

expression_string =  "("+ emp_d_matrix + "*(" + hov3_time + "<45))/" + str(total_emp)
new_matrix_name("mo" + emp_auto_mtx[2:], "access.emp.auto")
spec = common.get_mc_spec("mo" + emp_auto_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)

#15.	% of people+jobs accessible by road (SOV+HOV) in under 45 min by zone

expression_string =  "(("+ emp_d_matrix + " +" + pop_d_matrix + ")*(" + hov3_time + "<45))/" + str(total_emp+total_pop)
new_matrix_name("mo" + pop_emp_auto_mtx[2:], "access.pop.emp.auto")
spec = common.get_mc_spec("mo" + pop_emp_auto_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)

#16.	% of people accessible by active modes in under 45 min by zone

expression_string =  "("+ pop_d_matrix + "*((" + sov_dist + "*4)<45))/" + str(total_pop)
new_matrix_name("mo" + pop_active_mtx[2:], "access.pop.active")
spec = common.get_mc_spec("mo" + pop_active_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)

#17.	 % of jobs accessible by active modes in under 45 min by zone

expression_string =  "("+ emp_d_matrix + "*((" + sov_dist + "*4)<45))/" + str(total_emp)
new_matrix_name("mo" + emp_active_mtx[2:], "access.emp.active")
spec = common.get_mc_spec("mo" + emp_active_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)

#18.	% of people+jobs accessible by active modes in under 45 min by zone
expression_string =  "(("+ emp_d_matrix + " +" + pop_d_matrix + ")*((" + sov_dist + "*4)<45))/" + str(total_emp+total_pop)
new_matrix_name("mo" + pop_emp_active_mtx[2:], "access.pop.emp.active")
spec = common.get_mc_spec("mo" + pop_emp_active_mtx[2:], expression_string, None, "+", None, None, None)
matrix_calculator(spec)




{'average': 1.2e-05,
 'expression_type': 'FULL',
 'maximum': 0.002427,
 'maximum_at': {'destination': 1188, 'origin': 1018},
 'minimum': 0.0,
 'minimum_at': {'destination': 1018, 'origin': 1001},
 'num_evaluations': 12645136,
 'result_type': 'ORIGIN',
 'sum': 147.352676}

In [30]:
#HEATMAP -calculate average auto time

expression_string = "(" + sov_num + "*" + sov_time + "+" + hov2_num + "*" + hov2_time \
    + "+" + hov3_num + "*" + hov3_time+ ")"
denom_nums = [sov_num[2:],hov2_num[2:],hov3_num[2:]]

calc_avg_od (expression_string, denom_nums, avg_auto_time[2:], "avg.auto.time")

#HEATMAP - calculate average auto distance

expression_string = "(" + sov_num + "*" + sov_dist + "+" + hov2_num + "*" + hov2_dist \
    + "+" + hov3_num + "*" + hov3_dist+ ")"
denom_nums = [sov_num[2:],hov2_num[2:],hov3_num[2:]]

calc_avg_od (expression_string, denom_nums, avg_auto_dist[2:], "avg.auto.dist")


In [44]:
"(" + sov_num + "*" + sov_time + "+" + hov2_num + "*" + hov2_time \
    + "+" + hov3_num + "*" + hov3_time+ ")"

'(mf13*mf2+mf14*mf4+mf15*mf6)'

In [31]:

export_matrix_os = ["mo7","mo8","mo11","mo10","mo13","mo14","mo15","mo16","mo17","mo18","mo19",\
                    "mo21","mo22","mo23","mo24","mo25","mo26","mo27","mo28","mo29","mo30","mo31",\
                   "mo32","mo33","mo34","mo35","mo36"]
export_matrix_ds = ["md7","md8","md11","md10","md13","md14","md15","md16","md17","md18","md19",\
                    "md21","md22","md23","md24","md25","md26","md27"]


In [34]:
save_path = "C:\Users\mnasterska\Documents\Projects\GGH\MOE\HeatMap"
export_matrices(matrices = export_matrix_os,
                export_path = save_path + "\Source\\2041_C+2")


In [19]:
#Calculate average travel time per mode

#SOV
new_matrix = avg_sov_time

#check if results matrix exist
if not(eb.matrix(new_matrix)):
    create_matrix(matrix_id = new_matrix,
              matrix_name = "avg.sov.time",
            matrix_description = "avg.sov.time")

spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "(" + str(sov_num) + "*" + sov_time+ ")/((" + sov_num + "==0)+(" + sov_num + "))"
}

matrix_calculator(spec)

#HOV
new_matrix = avg_hov_time

#check if results matrix exist
if not(eb.matrix(new_matrix)):
    create_matrix(matrix_id = new_matrix,
              matrix_name = "avg.hov.time",
            matrix_description = "avg.hov.time")

spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "(" + hov2_num + "*" + hov2_time+ "+" + hov3_num + "*" + hov3_time+ \
    ")/(((" + hov2_num + "+" + hov3_num + ")==0)+(" + hov2_num + "+" + hov3_num + "))"
}

matrix_calculator(spec)

#Active
new_matrix = avg_active_time

#check if results matrix exist
if not(eb.matrix(new_matrix)):
    create_matrix(matrix_id = new_matrix,
              matrix_name = "avg.active.time",
            matrix_description = "avg.active.time")

spec = {
    "type": "MATRIX_CALCULATION",
    "result": new_matrix,
    "expression": "(" + hov2_num + "*" + hov2_time+ "+" + hov3_num + "*" + hov3_time+ \
    ")/(((" + hov2_num + "+" + hov3_num + ")==0)+(" + hov2_num + "+" + hov3_num + "))"
}

matrix_calculator(spec)


{'average': 4.932557,
 'expression_type': 'FULL',
 'maximum': 408.49707,
 'maximum_at': {'destination': 2087, 'origin': 7010},
 'minimum': 0.0,
 'minimum_at': {'destination': 1001, 'origin': 1001},
 'num_evaluations': 10751841,
 'result_type': 'FULL',
 'sum': 53034072.0}

In [ ]:
### Below might be unnecessary

In [11]:
export_matrix_os = ["mo28","mo29","mo30","mo31","mo32","mo33","mo34","mo35","mo36"]

In [16]:
export_matrix_os = ["mo7","md7","mo8","md8"]

In [27]:
#Heat Map - Aggregate matrices

matrices = os.listdir(save_path+"\Source")

import numpy as np
import csv

header = None
with open (os.path.join(save_path, "Aggregated_2016.csv"),"wb") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter = ",", quoting = csv.QUOTE_MINIMAL)
    firstrow = ["Zone"]
    aggregated = []
    for matrix in matrices:
        sums = False
        avgs = False
        matrix_num = os.path.splitext(matrix)[0]
        if matrix_num in sum_matrices:
            sums = True
        elif matrix_num in avg_matrices:
            avgs = True
        
        if sums:
            firstrow.append(sum_matrices[matrix_num])
        elif avgs:
            firstrow.append(avg_matrices[matrix_num])
        firstrow.append("")
        with open(os.path.join(save_path+"\Source", matrix),"r") as rawf:
            line = rawf.readline()
            if header == None:
                header = line.rstrip("\n").split(",")[1:]
            origin = [0 for row in range(len(header))]
            dest = [0 for row in range(len(header))]
            lines = rawf.readlines()

            for row in range(len(lines)):
                Curline = np.array(map(float,lines[row].rstrip("\n").split(",")[1:]))
                if sums:
                    origin[row] = sum(Curline)
                    for col in range(len(Curline)):
                        dest[col] += Curline[col]
                elif avgs:
                    origin[row] = Curline[Curline.nonzero()].mean()
                    for col in range(len(Curline)):
                        if Curline[col] != 0:
                            if dest[col] == 0:
                                dest[col] = [Curline[col],1]
                            else:
                                dest[col][0]+= Curline[col]
                                dest[col][1]+=1
        if avgs:
            for row in range (len(dest)):
                if dest[row] != 0:
                    dest[row] = dest[row][0]/dest[row][1]
        aggregated.append(origin)
        aggregated.append(dest)
    csvwriter.writerow(firstrow)
    for row  in range(len(header)):
        values = [header[row]]
        for col in range(len(aggregated)):
            values.append(aggregated[col][row])
        csvwriter.writerow(values)
print "Finished Aggregating MOEs"    

IndexError: list index out of range